In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
        
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1

        for s in size:
            num_features *= s
        return num_features

In [18]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [19]:
params = list(cnn.parameters()) #list of Hyper parameters
print(len(params))
n_weights = 0
n_biases = 0
for p in params:
    if len(p.size())== 1:
        desc = 'Bias'
        n_biases += 1
    else:
        desc = 'Weight'
        n_weights += 1
    print('{} : {}'.format(desc, p.size()))
print('\n' + '{} weights and {} biases in total'.format(n_weights, n_biases))

10
Weight : torch.Size([6, 1, 5, 5])
Bias : torch.Size([6])
Weight : torch.Size([16, 6, 5, 5])
Bias : torch.Size([16])
Weight : torch.Size([120, 400])
Bias : torch.Size([120])
Weight : torch.Size([84, 120])
Bias : torch.Size([84])
Weight : torch.Size([10, 84])
Bias : torch.Size([10])

5 weights and 5 biases in total


In [40]:
rand_img = torch.randn(1,1,32,32)
out = cnn(rand_img)
print(out)
print(out.max(), out.argmax())

tensor([[-0.0081, -0.0059, -0.0056,  0.0476,  0.1225, -0.1090, -0.0274, -0.0361,
         -0.0010, -0.0312]], grad_fn=<AddmmBackward>)
tensor(0.1225, grad_fn=<MaxBackward1>) tensor(4)


In [41]:
param = list(cnn.parameters())
param[0].size()

torch.Size([6, 1, 5, 5])

In [66]:
input = torch.randn(1,1, 32, 32)
cnn(input).shape #output of forward

out = cnn(input)

## Updating Parameters

In [67]:
print(out.shape)
target = torch.arange(1, 11)
target = target.view(1, -1)
target = target.type(torch.FloatTensor)

optimizer = optim.SGD(cnn.parameters(), lr=0.01)
optimizer.zero_grad()

criterion = nn.MSELoss()
loss = criterion(out, target)
loss.backward()
optimizer.step()

torch.Size([1, 10])
